In [ ]:
import os

from langchain_community.retrievers import (
    AzureAISearchRetriever,
)

from config_notebooks import *
from langchain.agents import AgentType, initialize_agent
from pprint import pprint

from langchain.chains import RetrievalQA
from langchain_core.tools import Tool
from langchain_core.pydantic_v1 import BaseModel, Field

from langchain.chat_models import AzureChatOpenAI

In [ ]:
files_list = ['AGL.pdf','coca-cola-business-sustainability-report-2022.pdf']
action_input = 'Scope 3 emissions'

In [ ]:

class DocumentInput(BaseModel):
    question: str = Field()
    
os.environ["AZURE_AI_SEARCH_SERVICE_NAME"] = AZURE_SEARCH_SERVICE_ENDPOINT
os.environ["AZURE_AI_SEARCH_INDEX_NAME"] = AZURE_SEARCH_INDEX_NAME
os.environ["AZURE_AI_SEARCH_API_KEY"] = AZURE_SEARCH_ADMIN_KEY
os.environ["AZURE_OPENAI_API_KEY"] =AZURE_OPENAI_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] =AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_VERSION"] = "2023-05-15"

In [ ]:

tools = []
llm = AzureChatOpenAI(azure_deployment=AZURE_OPENAI_DEPLOYMENT_GPT_4O_ID,
                      temperature=0)

for file in files_list:
    file_name = file.split('.')[0]
    retriever = AzureAISearchRetriever(
    content_key="content", top_k=3, index_name=AZURE_SEARCH_INDEX_NAME,
    filter="sourcepage eq '{}'".format(file_name))
    
    tool = Tool(
    args_schema=DocumentInput,
            name=file_name,
            description=f"useful when you want to answer questions about {file_name}",
            func=RetrievalQA.from_chain_type(llm=llm, retriever=retriever),
        )
    tools.append(tool)

agent = initialize_agent(
    agent=AgentType.OPENAI_FUNCTIONS,
    tools=tools,
    llm=llm,
    verbose=True,
)
action_input_prompt = "Compare " + action_input + " in "
for file in files_list:
    file_name = file.split('.')[0]
    action_input_prompt += file_name + ","   

pprint(action_input_prompt)
pprint("^^^^^^^^^^^^^^^^^^^^^^^^^^")
results = agent({"input": action_input_prompt})

In [ ]:

pprint(results["output"])